# Exploring UV extinction curves – Part 1 – SOLUTIONS

Solutions proposed for Part 1.

## Imports

The first step will be to import the libraries we will be using throughout this tutorial: 

- _Observations_ from _astroquery.mast_ to query the Barbara A. Mikulski Archive for Space Telescopes (MAST).
- _fits_ from _astropy.io_ for accessing FITS files
- _numpy_ for array manipulations

In [1]:
from astroquery.mast import Observations
from astropy.io import fits
import numpy as np

## Functions needed

Functions defined in the tutorial that will be used in the solutions of the exercises:

In [2]:
def changeNaN(wavv,fluxx,wavv1,fluxx1):
    
    """
    Removes negative flux values to avoid encountering NaN when converting to flux scale
    
    Inputs:
    :param wavv,wavv1: input wavelength arrays
    :type wavv,wavv1: float array
    :param fluxx,fluxx1: input wavelength arrays
    :type fluxx,fluxx1: float array
    
    Outputs: 
    :return: wavv,fluxx,wavv1,fluxx1 without the corresponding problematic values
    :rtype: float array
    """
    
    prob = np.where(fluxx < 0)[0]
    wavv = np.delete(wavv,prob)
    fluxx = np.delete(fluxx,prob)
    wavv1 = np.delete(wavv1,prob)
    fluxx1 = np.delete(fluxx1,prob)
    
    prob2 = np.where(fluxx1 < 0)[0]
    wavv = np.delete(wavv,prob2)
    fluxx = np.delete(fluxx,prob2)
    wavv1 = np.delete(wavv1,prob2)
    fluxx1 = np.delete(fluxx1,prob2)
    
    return wavv, fluxx, wavv1, fluxx1

In [3]:
def saveArray(array,filename):
    
    """
    Saves arrays into .txt files in current directory and checks that the files have been written properly
    
    Inputs:
    :param array: array to be saved to local directory
    :type array: float array
    :param filename: name of file to be saved
    :type filename: string
    
    Outputs: 
    :return: none
    :rtype: none
    """
    
    # First display the start and ending of the array to check later
    print('First 3 elements of original array:\n', array[:3])
    print('Last 3 elements of original array:\n', array[-3:])

    # Save the array in a text file
    np.savetxt(filename,array)
    
    # Display the saved content to check it was correctly written to the .txt file
    content = np.loadtxt(filename)
    print('First 3 elements of '+filename+':\n', content[:3])
    print('Last 3 elements of '+filename+':\n', content[-3:],'\n')
    
    return None

## Exercises

Now you can try to do it yourself! Try to obtain the extinction curve of a Large Magellanic Cloud (LMC) following the steps presented for the SMC one and look for the differences in the trend between the two.

### 1. Load the data

The targets for this exercise will be:

- Sk -69 206 (Observation IDs: swp36552, lwp15751) (reddened star)
- Sk -67 5 (Observation IDs: swp04827, lwr04170) (unreddened star)

First, do the database search for the reddened star:

In [4]:
IUEobs_sk_69_206 = Observations.query_criteria(objectname="SK-69206",obs_collection="IUE")
data_products_sk_69_206 = Observations.get_product_list(IUEobs_sk_69_206)

data_products_sk_69_206

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str6,str3,str8,str8,str56,str1,str62,str9,str28,str56,str1,str1,str1,str5,str20,int64,str6,str6,int64
276552,IUE,spectrum,lwp03703,ELBLL,S,mast:IUE/url/pub/iue/data/lwp/03000/lwp03703.elbll.gz,AUXILIARY,--,--,--,--,--,WRGCG,lwp03703.elbll.gz,100355,276552,PUBLIC,2
276552,IUE,spectrum,lwp03703,LILO,S,mast:IUE/url/pub/iue/data/lwp/03000/lwp03703.lilo.gz,AUXILIARY,--,--,--,--,--,WRGCG,lwp03703.lilo.gz,535333,276552,PUBLIC,2
276552,IUE,spectrum,lwp03703,MELOL,S,mast:IUE/url/pub/iue/data/lwp/03000/lwp03703.melol.gz,AUXILIARY,--,--,--,--,--,WRGCG,lwp03703.melol.gz,12071,276552,PUBLIC,2
276552,IUE,spectrum,lwp03703,RAW,S,mast:IUE/url/pub/iue/data/lwp/03000/lwp03703.raw.gz,AUXILIARY,--,--,--,--,--,WRGCG,lwp03703.raw.gz,358821,276552,PUBLIC,2
276552,IUE,spectrum,lwp03703,RILO,S,mast:IUE/url/pub/iue/data/lwp/03000/lwp03703.rilo.gz,AUXILIARY,--,--,--,--,--,WRGCG,lwp03703.rilo.gz,363834,276552,PUBLIC,2
276552,IUE,spectrum,lwp03703,SILO,S,mast:IUE/url/pub/iue/data/lwp/03000/lwp03703.silo.gz,AUXILIARY,--,--,--,--,--,WRGCG,lwp03703.silo.gz,89362,276552,PUBLIC,2
276552,IUE,spectrum,lwp03703,Preview-Full,S,mast:IUE/url/browse/previews/iue/mx/lwp/03000/gif/lwp03703.gif,PREVIEW,--,--,--,--,--,WRGCG,lwp03703.gif,10823,276552,PUBLIC,2
276552,IUE,spectrum,lwp03703,MXLO,S,mast:IUE/url/pub/iue/data/lwp/03000/lwp03703.mxlo.gz,SCIENCE,Minimum Recommended Products,--,--,--,--,WRGCG,lwp03703.mxlo.gz,18011,276552,PUBLIC,2
276552,IUE,spectrum,lwp03703,(extracted spectra/vo spectral container/SSAP) fits file,S,mast:IUE/url/pub/vospectra/iue2/lwp03703mxlo_vo.fits,SCIENCE,Minimum Recommended Products,(extracted spectra/vo spectral container/SSAP) fits file,--,--,--,WRGCG,lwp03703mxlo_vo.fits,48960,276552,PUBLIC,2


Narrow down the possibilities to the desired observation, observation ID: "swp36552":

In [5]:
IUEobs_sk_69_206_sw = Observations.query_criteria(obs_collection="IUE", obs_id="swp36552")
data_products_sk_69_206_sw = Observations.get_product_list(IUEobs_sk_69_206_sw)
yourProd_sk_69_206_sw = Observations.filter_products(data_products_sk_69_206_sw, extension="swp36552mxlo_vo.fits")

yourProd_sk_69_206_sw

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str6,str3,str8,str8,str56,str1,str62,str9,str28,str56,str1,str1,str1,str5,str20,int64,str6,str6,int64
327806,IUE,spectrum,swp36552,(extracted spectra/vo spectral container/SSAP) fits file,S,mast:IUE/url/pub/vospectra/iue2/swp36552mxlo_vo.fits,SCIENCE,Minimum Recommended Products,(extracted spectra/vo spectral container/SSAP) fits file,--,--,--,OBLCG,swp36552mxlo_vo.fits,48960,327806,PUBLIC,2


Now, download the data as before:

In [6]:
Observations.download_products(yourProd_sk_69_206_sw, mrp_only = False, cache = False) 

Local Path,Status,Message,URL
str48,str8,object,object
./mastDownload/IUE/swp36552/swp36552mxlo_vo.fits,COMPLETE,None,None


And again, in order to cover a wider wavelength range, let's also download the other part of the spectra using the second observation ID provided, observation ID: "lwp15751":

In [7]:
IUEobs_sk_69_206_lw = Observations.query_criteria(obs_collection="IUE", obs_id="lwp15751")
data_products_sk_69_206_lw = Observations.get_product_list(IUEobs_sk_69_206_lw)
yourProd_sk_69_206_lw = Observations.filter_products(data_products_sk_69_206_lw, extension="lwp15751mxlo_vo.fits")
Observations.download_products(yourProd_sk_69_206_lw, mrp_only = False, cache = False) 

Local Path,Status,Message,URL
str48,str8,object,object
./mastDownload/IUE/lwp15751/lwp15751mxlo_vo.fits,COMPLETE,None,None


Now, let's do the same for the unreddened star:

In [8]:
IUEobs_sk_67_5 = Observations.query_criteria(objectname="SK-675",obs_collection="IUE")
data_products_sk_67_5 = Observations.get_product_list(IUEobs_sk_67_5)

data_products_sk_67_5

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str6,str3,str8,str8,str56,str1,str62,str9,str28,str56,str1,str1,str1,str5,str20,int64,str6,str6,int64
293177,IUE,spectrum,lwp29080,ELBLL,S,mast:IUE/url/pub/iue/data/lwp/29000/lwp29080.elbll.gz,AUXILIARY,--,--,--,--,--,PHCAL,lwp29080.elbll.gz,193068,293177,PUBLIC,2
293177,IUE,spectrum,lwp29080,LILO,S,mast:IUE/url/pub/iue/data/lwp/29000/lwp29080.lilo.gz,AUXILIARY,--,--,--,--,--,PHCAL,lwp29080.lilo.gz,508441,293177,PUBLIC,2
293177,IUE,spectrum,lwp29080,MELOL,S,mast:IUE/url/pub/iue/data/lwp/29000/lwp29080.melol.gz,AUXILIARY,--,--,--,--,--,PHCAL,lwp29080.melol.gz,11897,293177,PUBLIC,2
293177,IUE,spectrum,lwp29080,RAW,S,mast:IUE/url/pub/iue/data/lwp/29000/lwp29080.raw.gz,AUXILIARY,--,--,--,--,--,PHCAL,lwp29080.raw.gz,341374,293177,PUBLIC,2
293177,IUE,spectrum,lwp29080,RILO,S,mast:IUE/url/pub/iue/data/lwp/29000/lwp29080.rilo.gz,AUXILIARY,--,--,--,--,--,PHCAL,lwp29080.rilo.gz,346716,293177,PUBLIC,2
293177,IUE,spectrum,lwp29080,SILO,S,mast:IUE/url/pub/iue/data/lwp/29000/lwp29080.silo.gz,AUXILIARY,--,--,--,--,--,PHCAL,lwp29080.silo.gz,84716,293177,PUBLIC,2
293177,IUE,spectrum,lwp29080,Preview-Full,S,mast:IUE/url/browse/previews/iue/mx/lwp/29000/gif/lwp29080.gif,PREVIEW,--,--,--,--,--,PHCAL,lwp29080.gif,5823,293177,PUBLIC,2
293177,IUE,spectrum,lwp29080,MXLO,S,mast:IUE/url/pub/iue/data/lwp/29000/lwp29080.mxlo.gz,SCIENCE,Minimum Recommended Products,--,--,--,--,PHCAL,lwp29080.mxlo.gz,17660,293177,PUBLIC,2
293177,IUE,spectrum,lwp29080,(extracted spectra/vo spectral container/SSAP) fits file,S,mast:IUE/url/pub/vospectra/iue2/lwp29080mxlo_vo.fits,SCIENCE,Minimum Recommended Products,(extracted spectra/vo spectral container/SSAP) fits file,--,--,--,PHCAL,lwp29080mxlo_vo.fits,48960,293177,PUBLIC,2


- Observation ID: "swp04827":

In [9]:
data_products_sk_67_5[42]

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str6,str3,str8,str8,str56,str1,str62,str9,str28,str56,str1,str1,str1,str5,str20,int64,str6,str6,int64
308097,IUE,spectrum,swp04827,(extracted spectra/vo spectral container/SSAP) fits file,S,mast:IUE/url/pub/vospectra/iue2/swp04827mxlo_vo.fits,SCIENCE,Minimum Recommended Products,(extracted spectra/vo spectral container/SSAP) fits file,--,--,--,MSJDW,swp04827mxlo_vo.fits,48960,308097,PUBLIC,2


In [10]:
IUEobs_sk_67_5_sw = Observations.query_criteria(obs_collection="IUE", obs_id="swp04827")
data_products_sk_67_5_sw = Observations.get_product_list(IUEobs_sk_67_5_sw)
yourProd_sk_67_5_sw = Observations.filter_products(data_products_sk_67_5_sw, extension="swp04827mxlo_vo.fits")
Observations.download_products(yourProd_sk_67_5_sw, mrp_only = False, cache = False) 

Local Path,Status,Message,URL
str48,str8,object,object
./mastDownload/IUE/swp04827/swp04827mxlo_vo.fits,COMPLETE,None,None


- Observation ID: "lwr04170"

In [11]:
data_products_sk_67_5[17]

obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
str6,str3,str8,str8,str56,str1,str62,str9,str28,str56,str1,str1,str1,str5,str20,int64,str6,str6,int64
296811,IUE,spectrum,lwr04170,(extracted spectra/vo spectral container/SSAP) fits file,S,mast:IUE/url/pub/vospectra/iue2/lwr04170mxlo_vo.fits,SCIENCE,Minimum Recommended Products,(extracted spectra/vo spectral container/SSAP) fits file,--,--,--,MSJDW,lwr04170mxlo_vo.fits,48960,296811,PUBLIC,2


In [12]:
IUEobs_sk_67_5_lw = Observations.query_criteria(obs_collection="IUE", obs_id="lwr04170")
data_products_sk_67_5_lw = Observations.get_product_list(IUEobs_sk_67_5_lw)
yourProd_sk_67_5_lw = Observations.filter_products(data_products_sk_67_5_lw, extension="lwr04170mxlo_vo.fits")
Observations.download_products(yourProd_sk_67_5_lw, mrp_only = False, cache = False) 

Local Path,Status,Message,URL
str48,str8,object,object
./mastDownload/IUE/lwr04170/lwr04170mxlo_vo.fits,COMPLETE,None,None


### 2. Open the fits file and explore it

In [13]:
filename_sk_69_206_sw = "./mastDownload/IUE/swp36552/swp36552mxlo_vo.fits"
fits.info(filename_sk_69_206_sw)

with fits.open(filename_sk_69_206_sw) as hdulist:
    spectrum_sk_69_206_sw = hdulist[1].data

wav_sk_69_206_sw = spectrum_sk_69_206_sw[0][0] # A
flux_sk_69_206_sw = spectrum_sk_69_206_sw[0][1] # ergs/cm2/sec/A

filename_sk_69_206_lw = "./mastDownload/IUE/lwp15751/lwp15751mxlo_vo.fits"
fits.info(filename_sk_69_206_lw)

with fits.open(filename_sk_69_206_lw) as hdulist:
    spectrum_sk_69_206_lw = hdulist[1].data

wav_sk_69_206_lw = spectrum_sk_69_206_lw[0][0] # A
flux_sk_69_206_lw = spectrum_sk_69_206_lw[0][1] # ergs/cm2/sec/A

filename_sk_67_5_sw = "./mastDownload/IUE/swp04827/swp04827mxlo_vo.fits"
fits.info(filename_sk_67_5_sw)

with fits.open(filename_sk_67_5_sw) as hdulist:
    spectrum_sk_67_5_sw = hdulist[1].data

wav_sk_67_5_sw = spectrum_sk_67_5_sw[0][0] # A
flux_sk_67_5_sw = spectrum_sk_67_5_sw[0][1] # ergs/cm2/sec/A

filename_sk_67_5_lw = "./mastDownload/IUE/lwr04170/lwr04170mxlo_vo.fits"
fits.info(filename_sk_67_5_lw)

with fits.open(filename_sk_67_5_lw) as hdulist:
    spectrum_sk_67_5_lw = hdulist[1].data

wav_sk_67_5_lw = spectrum_sk_67_5_lw[0][0] # A
flux_sk_67_5_lw = spectrum_sk_67_5_lw[0][1] # ergs/cm2/sec/A

wav_sk_69_206_sw, flux_sk_69_206_sw, wav_sk_67_5_sw, flux_sk_67_5_sw = changeNaN(wav_sk_69_206_sw, flux_sk_69_206_sw, wav_sk_67_5_sw, flux_sk_67_5_sw)
wav_sk_69_206_lw, flux_sk_69_206_lw, wav_sk_67_5_lw, flux_sk_67_5_lw = changeNaN(wav_sk_69_206_lw, flux_sk_69_206_lw, wav_sk_67_5_lw, flux_sk_67_5_lw)

Filename: ./mastDownload/IUE/swp36552/swp36552mxlo_vo.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     351   ()      
  1  Spectral Container    1 BinTableHDU    141   1R x 4C   [495E, 495E, 495E, 495I]   
Filename: ./mastDownload/IUE/lwp15751/lwp15751mxlo_vo.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     350   ()      
  1  Spectral Container    1 BinTableHDU    141   1R x 4C   [563E, 563E, 563E, 563I]   
Filename: ./mastDownload/IUE/swp04827/swp04827mxlo_vo.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     345   ()      
  1  Spectral Container    1 BinTableHDU    141   1R x 4C   [495E, 495E, 495E, 495I]   
Filename: ./mastDownload/IUE/lwr04170/lwr04170mxlo_vo.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     353   ()      
  1  Spectral Container    1 BinTableHDU    141 

### 3. Save the data to your local computer

In [14]:
arrays = [wav_sk_69_206_sw, flux_sk_69_206_sw, wav_sk_67_5_sw, flux_sk_67_5_sw, wav_sk_69_206_lw, flux_sk_69_206_lw, wav_sk_67_5_lw, flux_sk_67_5_lw]
filenames = ['wav_sk_69_206_sw.txt', 'flux_sk_69_206_sw.txt', 'wav_sk_67_5_sw.txt', 'flux_sk_67_5_sw.txt', 'wav_sk_69_206_lw.txt', 'flux_sk_69_206_lw.txt', 'wav_sk_67_5_lw.txt', 'flux_sk_67_5_lw.txt']

for i in range(len(arrays)):
    saveArray(arrays[i],filenames[i])

First 3 elements of original array:
 [1150.5802 1152.2566 1153.933 ]
Last 3 elements of original array:
 [1975.3384 1977.0146 1978.691 ]
First 3 elements of wav_sk_69_206_sw.txt:
 [1150.5802002 1152.2565918 1153.9329834]
Last 3 elements of wav_sk_69_206_sw.txt:
 [1975.33837891 1977.01464844 1978.69104004] 

First 3 elements of original array:
 [2.7181413e-14 3.0124900e-14 7.0772984e-14]
Last 3 elements of original array:
 [8.4576599e-14 8.2177856e-14 8.2617500e-14]
First 3 elements of flux_sk_69_206_sw.txt:
 [2.71814126e-14 3.01248996e-14 7.07729841e-14]
Last 3 elements of flux_sk_69_206_sw.txt:
 [8.45765989e-14 8.21778558e-14 8.26174998e-14] 

First 3 elements of original array:
 [1150.5802 1152.2566 1153.933 ]
Last 3 elements of original array:
 [1975.3384 1977.0146 1978.691 ]
First 3 elements of wav_sk_67_5_sw.txt:
 [1150.5802002 1152.2565918 1153.9329834]
Last 3 elements of wav_sk_67_5_sw.txt:
 [1975.33837891 1977.01464844 1978.69104004] 

First 3 elements of original array:
 [2.34

## Additional Resources

For more information about the MAST archive and details about mission data: 

<a href="https://mast.stsci.edu/api/v0/index.html">MAST API</a> <br>
<a href="https://archive.stsci.edu/iue/">International Ultraviolet Explorer Page (MAST)</a> <br>
<a href="https://archive.stsci.edu/iue/mdr_help.html">IUE Data Retrieval Help Page</a> 

For more information about extinction curves and their parametrization:


<a href="https://ui.adsabs.harvard.edu/abs/2003ApJ...594..279G/abstract">A quantitative comparison of SMC, LMC and Milky Way UV to NIR extinction curves</a> <br>
<a href="https://adsabs.harvard.edu/pdf/1990ApJS...72..163F">An analysis of the shapes of the ultraviolet extinction curves. III. An atlas of ultraviolet extinction curves</a> 

## About this Notebook

**Author**: Clara Puerto Sánchez <br>
**Updated on**: 06/07/2022

## Citations

If you use `astropy`, `numpy`, `scipy` or `matplotlib` for published research, please cite the
authors. Follow these links for more information about citing them:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `numpy`](https://numpy.org/citing-numpy/)
* [Citing `scipy`](https://scipy.org/citing-scipy/)
* [Citing `matplotlib`](https://matplotlib.org/stable/users/project/citing.html)

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>